In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from timeseries import *
from models import *
import pickle 
import numpy as np
from sklearn.model_selection import train_test_split
from fastai.distributed import *
import neptune
from neptunecontrib.monitoring.fastai import NeptuneMonitor
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

/opt/anaconda/envs/pytorch/lib/python3.7/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


In [3]:
scale_type = 'normalize'
scale_by_channel = False
scale_by_sample  = True 
scale_range = (-1, 1)
act_fn = 'relu'
nf = 64
bs = 64
pool = nn.AdaptiveAvgPool1d

# Model

In [4]:
import torch
import torch.nn as nn
from models.layers import *

# def convlayer(c_in,c_out,ks=3,padding='same',bias=True,stride=1,
#               bn_init=False,zero_bn=False,bn_before=True,
#               act_fn='relu', dilation=1, **kwargs):
#     '''conv layer (padding="same") + bn + act'''
#     if ks % 2 == 1 and padding == 'same': padding = ks // 2
#     layers = [ConvSP1d(c_in,c_out, ks, bias=bias, stride=stride) if padding == 'same' else \
#     nn.Conv1d(c_in,c_out, ks, stride=stride, padding=padding, bias=bias, dilation=dilation)]
#     bn = GhostBatchNorm(num_features=c_out,num_splits=16)
#     if bn_init: nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
#     if bn_before: layers.append(bn)
#     if act_fn: layers.append(get_act_layer(act_fn, **kwargs))
#     if not bn_before: layers.append(bn)
#     return nn.Sequential(*layers)

class GhostBatchNorm(nn.BatchNorm1d):
    def __init__(self, num_features, num_splits, eps=1e-05, momentum=0.1, weight_freeze=False, bias_freeze=False, weight_init=1.0, bias_init=0.0):
        super().__init__(num_features, eps=eps, momentum=momentum)
        self.num_splits = num_splits
        if weight_init is not None: self.weight.data.fill_(weight_init)
        if bias_init is not None: self.bias.data.fill_(bias_init)
        self.weight.requires_grad = not weight_freeze
        self.bias.requires_grad = not bias_freeze
        self.register_buffer('running_mean', torch.zeros(num_features*self.num_splits))
        self.register_buffer('running_var', torch.ones(num_features*self.num_splits))

    def train(self, mode=True):
        if (self.training is True) and (mode is False): #lazily collate stats when we are going to use them
            self.running_mean = torch.mean(self.running_mean.view(self.num_splits, self.num_features), dim=0).repeat(self.num_splits)
            self.running_var = torch.mean(self.running_var.view(self.num_splits, self.num_features), dim=0).repeat(self.num_splits)
        return super().train(mode)
        
    def forward(self, input):
        N, C, H = input.shape
        if self.training or not self.track_running_stats:
            return nn.functional.batch_norm(
                input.view(-1, C*self.num_splits, H), self.running_mean, self.running_var, 
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H) 
        else:
            return nn.functional.batch_norm(
                input, self.running_mean[:self.num_features], self.running_var[:self.num_features], 
                self.weight, self.bias, False, self.momentum, self.eps)

class SeModule(nn.Module):
    def __init__(self, ch, reduction, act_fn='relu'):
        super().__init__()
        nf = math.ceil(ch//reduction/8)*8
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.conv1 = convlayer(ch, nf, 1, act_fn=act_fn)
        self.conv2 = convlayer(nf, ch, 1, act_fn=False)
        
    def forward(self, x):
        res = self.pool(x)
        res = self.conv1(res)
        res = self.conv2(res)
        res = nn.functional.sigmoid(res)
        return x * res
 
       
class ResBlock(nn.Module):
    def __init__(self, ni, nf, ks=[7, 5, 3], act_fn='relu'):
        super().__init__()
        self.conv1 = convlayer(ni, nf, ks[0], act_fn=act_fn)
        self.conv2 = convlayer(nf, nf, ks[1], act_fn=act_fn)
        self.conv3 = convlayer(nf, nf, ks[2], act_fn=False)
        self.se = SeModule(nf, 16, act_fn=act_fn)
        # expand channels for the sum if necessary
        self.shortcut = noop if ni == nf else convlayer(ni, nf, ks=1, act_fn=False)
        self.act_fn = get_act_layer(act_fn)

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # Squeeze and Excitation
        x = self.se(x)
        # Shortcut
        sc = self.shortcut(res)
        # Residual
        x += sc
        x = self.act_fn(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self,c_in, c_out, nf=64, pool=nn.AdaptiveAvgPool1d, act_fn='relu'):
        super().__init__()
        self.block1 = ResBlock(c_in, nf, ks=[7, 5, 3], act_fn=act_fn)
        self.block2 = ResBlock(nf, nf * 2, ks=[7, 5, 3], act_fn=act_fn)
        self.block3 = ResBlock(nf * 2, nf * 2, ks=[7, 5, 3], act_fn=act_fn)
        self.gap = pool(1)
        self.fc = nn.Linear(nf * 2, c_out)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

# Training

In [5]:
from fastai.utils.mod_display import *
def _resnet_split(m:nn.Module): return (m.block1, m.block2, m.block3, m.fc)
def train_and_predict(x, y, x_test):
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42123456)
    data = (ItemLists(Path("data"), TSList(x_train),TSList(x_val))
        .label_from_lists(y_train, y_val)
        .add_test(TSList(x_test))
        .databunch(bs=bs, val_bs=bs * 2)
        .scale(scale_type=scale_type, scale_by_channel=scale_by_channel, 
             scale_by_sample=scale_by_sample,scale_range=scale_range)
    )
    model = ResNet(data.features, data.c, act_fn=act_fn, nf=nf, pool=pool)
    kappa = KappaScore()
    loss_func=LabelSmoothingCrossEntropy()
    learn = Learner(data, model, metrics=[accuracy], loss_func=loss_func, opt_func=Ranger).to_fp16()
    learn.split(_resnet_split)
    learn.unfreeze()
    with progress_disabled_ctx(learn) as learn:
        learn.fit_one_cycle(100)
    learn.freeze()
    with progress_disabled_ctx(learn) as learn:
        learn.fit_one_cycle(50)
    print(learn.validate())
    preds,y = learn.get_preds(ds_type = DatasetType.Test)
    learn.destroy()
    return list(preds.argmax(dim=-1))

In [6]:
files = Path("Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow").ls()
files

[PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P08T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P07E.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P04T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P01T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P02E.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P03T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P05T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P01E.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P04E.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow/parsed_P07T.mat'),
 PosixPath('Clinical-Brain-Computer-Interfaces-Challenge-WCC

In [7]:
subjects = {}

In [8]:
from scipy.io import loadmat
for f in files:
    if "mat" not in str(f):
        continue
        
    sn = f.stem.split("_")[-1][:3]
    if sn not in subjects:
        subjects[sn]={'train': {'x': [], 'y': []}, 'test': {'x':[]}}
        
    if "T.mat" in str(f):
        annots = loadmat(f)
        subjects[sn]['train']['x'] += annots['RawEEGData'].tolist()
        subjects[sn]['train']['y'] += np.stack(annots['Labels'],axis=1)[0].tolist()
    
    elif "E.mat" in str(f):
        annots = loadmat(f)
        subjects[sn]['test']['x'] += annots['RawEEGData'].tolist()

In [9]:
subjects.keys()

dict_keys(['P08', 'P07', 'P04', 'P01', 'P02', 'P03', 'P05', 'P06', 'P09', 'P10'])

In [10]:
del subjects['P09']
del subjects['P10']
subjects.keys()

dict_keys(['P08', 'P07', 'P04', 'P01', 'P02', 'P03', 'P05', 'P06'])

In [11]:
with pd.ExcelWriter('Jin_Andry_WithinSubject.xlsx') as writer:
    for name, data in subjects.items():
        x = np.array(data['train']['x'])
        y = np.array(data['train']['y'])
        x_test = np.array(data['test']['x'])
        print(name)
        pred_list = train_and_predict(x, y, x_test)
        preds_json = [ {
            'Subject Name':name, 
            'Trial Index': i+1, 
            'Prediction': (p+1).cpu().numpy() 
        } for i, p in enumerate(pred_list) ]
        df = pd.DataFrame(preds_json)
        df.to_excel(writer, sheet_name=name, index=None)

P08


[0.5440176, tensor(1.)]


this Learner object self-destroyed - it still exists, but no longer usable
P07


[0.51252186, tensor(0.7500)]


this Learner object self-destroyed - it still exists, but no longer usable
P04


...

this Learner object self-destroyed - it still exists, but no longer usable
P05


[0.84325457, tensor(0.5000)]


this Learner object self-destroyed - it still exists, but no longer usable
P06


[0.5455694, tensor(0.8750)]


this Learner object self-destroyed - it still exists, but no longer usable
